## **CA2-AI**
# **Amir-Hossein Shahgholi(810199441)**

### Part1-Genetic

In first part we want to implementing a genetic algorithm for portfolio management:

At first we should create our Portfolio class to representation our chromosomes.

In [ ]:
class Portfolio:
    data = list
    fitness_result = int
    
    def __init__(self, data):
        self.data = data

In our Chromosome class we have a list of data that represent amount of each shares in our Protfolio.

Now we should create our population first. We fill data with some random numbers that sum of them will be 1.
To do this we first generate some random numbers and then normalize the numbers.</br>
Below function will do that for us.

In [ ]:
import random

ZERO_PERCENT = 0.6
def create_sample_data(number_of_data):
    numbers = []
    for _ in range(number_of_data):
        if (random.random()< ZERO_PERCENT):
            numbers.append(0)
        else:
            random_no = random.random()
            numbers.append(random_no)
    total = sum(numbers)
    numbers = [x/total for x in numbers]
    return numbers

So we create sample portfolio with randoms data that represent our population.
ٔNumber of Choromosomes in our population set as 1000.

In [ ]:
NUMBER_OF_CHOROMOSOME_IN_POPULATION = 1000
NUMBER_OF_SHARES = 400

population = []

for _ in range(NUMBER_OF_CHOROMOSOME_IN_POPULATION):
    sample_data = create_sample_data(NUMBER_OF_SHARES)
    temp_portfolio = Portfolio(data=sample_data)
    population.append(temp_portfolio)

In the next step we should define a fitness function to calculate a fitness value for each portfolio.
For fitness function first we find total_returns of our portfolio and then we should find risk of our portfolio using the portfolio's standard deviation and then we calculate a fitness value with this formula: **fitness = (portfolio_return - risk_penalty * portfolio_risk) / return_penalty**

In [ ]:
import numpy as np

risk_penalty = 0.6
return_penalty = 10


def calculate_fitness(returns: list, risks: list, portfolio):
    fitness = 0
    returns = np.array(returns, dtype=np.float64)
    risks = np.array(risks, dtype=np.float64)
    shares_percent = np.array(portfolio.data, dtype=np.float64)
    num_zeros = np.count_nonzero(shares_percent == 0.0)
    if (NUMBER_OF_SHARES - num_zeros) <30:
        fitness -= 0.1
    portfolio_return = np.dot(returns, shares_percent)
    portfolio_risk = np.sqrt(np.dot(shares_percent**2, risks))
    fitness += (portfolio_return - risk_penalty * portfolio_risk) / return_penalty
    return fitness

To find crossover of two chromosomes we use uniform crossover and then after finding child's data we normalize the data to have some of one in child's data. 

In [ ]:
def crossover(portfolio1, portfolio2) -> Portfolio:
    pdata1 = portfolio1.data 
    pdata2 = portfolio2.data
    child_data = []
    for i in range(NUMBER_OF_SHARES):
        if random.randint(0, 1) == 0:
            child_data.append(pdata1[i])
        else:
            child_data.append(pdata2[i])
    total = sum(child_data)
    child_data = [x/total for x in child_data]
    return Portfolio(child_data)

And we define a function for mutation that get portfolio data and replace the portfolio's data with some new data.

In [ ]:
def mutation(portfolio) -> Portfolio:
    portfolio.data = create_sample_data(len( portfolio.data))
    return portfolio

Read the sample data using `csv` library and save returns and risk of shares in two lists.

In [ ]:
import csv

returns = []
risks = []

with open('sample.csv', 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        returns.append(row['return'])
        risks.append(row['risk'])

We define a function to print data of portfolio.

In [ ]:
def print_portfolio_data(portfolio, returns, risks):
    returns = np.array(returns, dtype=np.float64)
    risks = np.array(risks, dtype=np.float64)
    shares_percent = np.array(portfolio.data, dtype=np.float64)
    portfolio_return = np.dot(returns, shares_percent)
    portfolio_risk = np.sqrt(np.dot(shares_percent**2, risks))
    print(f"Fitness: {portfolio.fitness_result}")
    print(f"Return: {portfolio_return}")
    print(f"Risk: {portfolio_risk}")
    print(f"Data: {portfolio.data}")

In this main step we create new generations. First we calculate the fitness for all choromosomes in our populationa and then we sort them, in next part we going into a loop that generate new generations in each loop.
In each loop we run the crossover function with probability of PC and mutation with probability of PM. After crossover we delete the choromosome with lowest fitness and then apend new choromosome to the population and when we got expected fitness we break the loop.

In [ ]:
PC = 0.9
PM = 0.05
for cal in population:
    cal.fitness_result = calculate_fitness(returns, risks, cal)
population = sorted(population, key=lambda portfolio: portfolio.fitness_result)
while True:
    if random.random() <= PC:
        population_no = len(population)
        parents = [population[i] for i in random.choices(range(population_no), weights=range(population_no), k=2)]
        new_child = crossover(parents[0], parents[1])
        new_child.fitness_result = calculate_fitness(returns, risks, new_child)
        population.pop(0)
        for z, chrom in enumerate(population):
            if chrom.fitness_result > new_child.fitness_result:
                population.insert(z, new_child)
                break
        else:
            population.append(new_child)
    if random.random() <= PM:
        random_chrom_index = random.randint(0,len(population)-1)
        population[random_chrom_index] = mutation(population[random_chrom_index])
    if population[-1].fitness_result > 1:
        break
print_portfolio_data(population[-1], returns, risks)
    

Fitness: 1.0007666126615837
Return: 10.227763423392163
Risk: 0.3668288279605435
Data: [0.011424351576230393, 0.0, 0.007879127725982002, 0.0, 0.0, 0.0, 0.0, 0.0, 0.12078337911515351, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0030441146315688017, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0006226355001908042, 0.0, 0.0024321722952422562, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0036543175598253447, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0010078069179856528, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.09668006950690515, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05588339140119659, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0002164497711016052, 0.0021540467683499893, 0.0, 0.0, 0.0, 0.0, 0.0, 0.000897766905020089, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0012512132039402405, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

### Answer a questions:

**1-** The small population can result in the algorithm getting stuck in a local optimum answer and not finding the global optimum answer. In large population finding the answer take long time and can result in longer computation times, which can be a problem for optimization problems with large scales.

**2-** It increases the computation times and the memory usage very much and unnecessary data(like choromosomes with low fitness) take large amount of memmory.

**3-** With crossover combining information from two or more parents to create better offsprings and mutation introducing random changes to the offspring to create some new genes. We need both of them for example crossover without mutation can getting stuck in a local answer and new childs don't getting better in new generations.

**4-** Chosing a good Fitness function, the initial population size and their data should be chosen carefully to create variant init data and parameters of the probability of crossover and mutation are helpful to make iit faster.

**5-** The algorithm may find local optimum answer and not finding the global optimum and that answer may not be good for our fitness. In this cases we can use mutation with more probability and also we can stop the loop if in new generation our fitness not growing.

**-6** As said in above question we can break the loop if we don't have grow in our new generation's fitness.

## Part2-Game